# DRIVE Dataset Retinal Vessels Segmentation

**Author:** Mohammad Ahadzadeh  
**Date:** April 3, 2025  
**Project:** Retinal Vessel Segmentation 

## Preprocessing the Images


### Extract the Green Channel
Retinal images often show vessels best in the green channel, so we will extract the green channel from the input images. This will help us to view and segment the vessels more clearly [1].


In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
image = cv2.imread("DRIVE/training/images/22_training.tif")
green_channel = image[:, :, 1] 


### Apply Contrast Limited Adaptive Histogram Equalization (CLAHE)

Using this method we can enhance the vessels-background contrast without amplifying noise [2].

In [3]:
clahe = cv2.createCLAHE(clipLimit=5.5, tileGridSize=(32,32))
enhanced = clahe.apply(green_channel)


### Noise Reduction and Vessel Enhancement

Using the Frangi filter from scikit-image is ideal for detecting tubular structures like vessels [3].



In [6]:
from skimage import img_as_float
from skimage.filters import frangi
enhanced_float = img_as_float(enhanced)
vesselness = frangi(enhanced_float, sigmas=(1, 15), scale_step=2, beta=5, gamma=0.04)


## Segmentation of Vessels
### Thresholding 
Otsu's method is a simple and effective method for thresholding vessels in retinal images [4].

In [7]:
from skimage.filters import threshold_otsu
thresh = threshold_otsu(vesselness)
binary = vesselness > thresh


### Morphological Operations

Using the closing operation to connect small objects and the opening operation to remove noise.

In [8]:
from skimage.morphology import closing, opening, disk
selem_close = disk(1)
cleaned = closing(binary, selem_close)

### Removing small objects

To remove small objects we will keep the connected components and remove components that are smaller than a certain threshold.


In [10]:
from skimage.morphology import remove_small_objects
min_size = 35
cleaned = remove_small_objects(cleaned, min_size=min_size)
segmented = (cleaned * 255).astype(np.uint8)


## Apply the Field of View Mask

In [11]:
import cv2
mask_path = "DRIVE/training/masks/22_training_mask.gif"
mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
_, mask_binary = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
segmented_masked = cv2.bitwise_and(segmented, segmented, mask=mask_binary)
cv2.imshow("Segmented Vessels", segmented_masked)
cv2.waitKey(0)

-1

## Implementation

This notebook only provides a detailed representation of this project. However, the actual execution of the segmentation pipeline, including file handling and the calculation of performance metrics across all data is written by a separate Python script located in the same directory as this notebook, named "DRIVE_RVS_Project.py".

## Tweaking Parameters

To optimize the segmentation pipeline, several parameters were adjusted based on visual inspection and quantitive results.

CLAHE Parameters:

I picked clipLimit=5.5 and tileGridSize=(32,32)`. These settings enhanced contrast and helped to recognizing vessels as much as possible but introduced some noise.
  
Frangi Filter Parameters:

The original sigmas=(1, 15) with scale_step=2 was effective, but finding beta=5 and gamma=0.04 by trials and errors took a while but I think I just found the right balance.


## Results

In [ ]:
Average Metrics:
Accuracy:    0.9419
Sensitivity: 0.6074
Specificity: 0.9737
Dice Coeff:  0.6430

Comparing to the similar approaches that utilized traditional image processing methods like me, my result seems to be decent, especially considering that this is my first experience in image processing field but there are a lot to improve. I'll keep learning and I might explore more advanced techniques in the future.

## References

“DRIVE Digital Retinal Images for Vessel Extraction.” Accessed: Apr. 03, 2025. [Online]. Available: https://www.kaggle.com/datasets/andrewmvd/drive-digital-retinal-images-for-vessel-extraction

[1]	A. Khanal and R. Estrada, “Dynamic Deep Networks for Retinal Vessel Segmentation,” Front. Comput. Sci., vol. 2, Aug. 2020, doi: 10.3389/fcomp.2020.00035.

[2]	N. Memari, A. R. Ramli, M. I. B. Saripan, S. Mashohor, and M. Moghbel, “Retinal Blood Vessel Segmentation by Using Matched Filtering and Fuzzy C-means Clustering with Integrated Level Set Method for Diabetic Retinopathy Assessment,” J. Med. Biol. Eng., vol. 39, no. 5, pp. 713–731, Oct. 2019, doi: 10.1007/s40846-018-0454-2.

[3]	S. Mahapatra, S. Agrawal, P. K. Mishro, and R. B. Pachori, “A novel framework for retinal vessel segmentation using optimal improved frangi filter and adaptive weighted spatial FCM,” Comput. Biol. Med., vol. 147, p. 105770, Aug. 2022, doi: 10.1016/j.compbiomed.2022.105770.

[4]	F. Tian, Y. Li, J. Wang, and W. Chen, “Blood Vessel Segmentation of Fundus Retinal Images Based on Improved Frangi and Mathematical Morphology,” Comput. Math. Methods Med., vol. 2021, p. 4761517, May 2021, doi: 10.1155/2021/4761517.